In [ ]:
#Задание 1

In [42]:
import pandas as pd
log = pd.read_csv('C:\\dz4\\visit_log.csv', sep=';')
#Сначала создал столбец и заполнил источником, а потом фильтрами последовательно прогнал условия
#Может можно и более изящно сделать
log['source_type'] = log['traffic_source']
log.loc[log.traffic_source.isin(['google', 'yandex']), 'source_type'] = 'organic'
log.loc[log.traffic_source.isin(['paid', 'email']) & log.region.isin(['Russia']), 'source_type'] = 'ad'
log.loc[log.traffic_source.isin(['paid', 'email']) & ~log.region.isin(['Russia']), 'source_type'] = 'other'
#log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [ ]:
#Задание 2

In [91]:
url = pd.read_csv('C:\\dz4\\URLs.txt')
url.url.str.contains('\d{8}-')
#Как то совсем просто

0     False
1     False
2     False
3      True
4      True
      ...  
88     True
89     True
90     True
91     True
92     True
Name: url, Length: 93, dtype: bool

In [ ]:
#Задание 3

In [106]:
ratings = pd.read_csv('C:\\ml-25m\\ratings.csv')
#Первым шагом оставляем список пользователей, которые оставили более 100 оценок
rate_count = ratings.pivot_table(index = 'userId', values = 'timestamp', aggfunc = 'count', fill_value = 0).reset_index()
rate_count = rate_count.loc[rate_count.timestamp > 100].drop('timestamp', axis = 1)
#расчитываем max и min значения timestamp и добавляем их к списку отфильтрованных пользователей
max_ts = ratings.pivot_table(index = 'userId', values = 'timestamp', aggfunc = 'max', fill_value = 0).reset_index()
min_ts = ratings.pivot_table(index = 'userId', values = 'timestamp', aggfunc = 'min', fill_value = 0).reset_index()
user_lifecycle = rate_count.join(max_ts, rsuffix = '_max').join(min_ts, rsuffix = '_min').drop(['userId_max', 'userId_min'], axis = 1)
#жизненный цикл в секундах
user_lifecycle['lifecycle_sec'] = (user_lifecycle['timestamp'] - user_lifecycle['timestamp_min'])
#удаление лишних столбцов
user_lifecycle.drop(['timestamp', 'timestamp_min'], axis = 1)

,userId,lifecycle_sec
1,2,2499
2,3,126619684
3,4,258010
4,5,27842266
7,8,790488
...,...,...
162532,162533,48682606
162533,162534,23204127
162536,162537,1086
162537,162538,4871


In [ ]:
#Задание 4

In [110]:
#Входные данные
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [121]:
#Решение

#часть 1
air.merge(auto, how='outer').merge(rzd, how='outer').fillna(0)

#часть 2
client_base.merge(air, how='outer').merge(auto, how='outer').merge(rzd, how='outer').fillna(0)#.drop('address', axis = 1)
#часть1 можно получить добавив (сняв коммент) .drop('address', axis = 1)
#

,client_id,air_revenue,auto_revenue,rzd_revenue
0,115,81.0,912.0,981.0
1,116,4.0,4834.0,0.0
2,117,13.0,98.0,0.0
3,118,173.0,0.0,0.0
4,113,0.0,57483.0,10283.0
5,114,0.0,83.0,5774.0
6,111,0.0,0.0,1093.0
7,112,0.0,0.0,2810.0
